In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install opencv-python

In [ ]:
pip install dlib

In [ ]:
import dlib
import cv2
import os

# Some constants
RESULT_PATH = 'your result path'       # The path that the result images will be saved
IMG_PATH = 'your img path'      # Dataset path
LOG_PATH = 'your log path'            # The path for the working log file
LIP_MARGIN = 0.3                # Marginal rate for lip-only image.
RESIZE = (64,64)                # Final image size
logfile = open(LOG_PATH,'w')
# Face detector and landmark detector
face_detector = dlib.get_frontal_face_detector()
landmark_detector = dlib.shape_predictor("path of your shape_predictor_68_face_landmarks.dat")

In [ ]:
def shape_to_list(shape):
    coords = []
    for i in range(0, 68):
        coords.append((shape.part(i).x, shape.part(i).y))

    return coords

def process_image(image_path):
    # Read image
    frame = cv2.imread(image_path)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert image into grayscale

    # Obtain face landmark information
    face_rects = face_detector(gray, 1)  # Detect face
    if len(face_rects) != 1:
        print("Invalid number of faces detected in:", image_path)
        logfile.write(image_path + " : Invalid number of faces detected \r\n")
        return

    rect = face_rects[0]  # Proper number of faces
    landmark = landmark_detector(gray, rect)  # Detect face landmarks
    landmark = shape_to_list(landmark)

    # Crop images
    lip_landmark = landmark[48:68]  # Landmark corresponding to lip
    lip_x = sorted(lip_landmark, key=lambda pointx: pointx[0])  # Lip landmark sorted for determining lip region
    lip_y = sorted(lip_landmark, key=lambda pointy: pointy[1])
    x_add = int((-lip_x[0][0] + lip_x[-1][0]) * LIP_MARGIN)  # Determine Margins for lip-only image
    y_add = int((-lip_y[0][1] + lip_y[-1][1]) * LIP_MARGIN)
    crop_pos = (lip_x[0][0] - x_add, lip_x[-1][0] + x_add, lip_y[0][1] - y_add, lip_y[-1][1] + y_add)  # Crop image
    cropped = frame[crop_pos[2]:crop_pos[3], crop_pos[0]:crop_pos[1]]
    cropped = cv2.resize(cropped, (RESIZE[0], RESIZE[1]), interpolation=cv2.INTER_CUBIC)  # Resize

    # Save result
    '''
    filename = os.path.basename(image_path)
    result_path = os.path.join(RESULT_PATH, filename)
    cv2.imwrite(result_path, cropped)  # Write lip image
    print("Processed:", image_path)

    '''
    # Determine relative path from root folder
    relative_path = os.path.relpath(image_path, IMG_PATH)
    result_path = os.path.join(RESULT_PATH, relative_path)

    # Create directories if they don't exist
    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    # Save result
    cv2.imwrite(result_path, cropped)  # Write lip image
    print("Processed:", image_path)



In [ ]:
# Process all images in the directory
'''
MIRACL-VC1
  ㄴF01
    ㄴwords
      ㄴ01(label)
        ㄴ01(instance)
          ㄴcolor_001.jpg
                 :
          ㄴcolor_007.jpg
                 :
          ㄴdepth_007.png
        ㄴ02(instance)
        ㄴ03(instance)
                :
        ㄴ10(instance)
      ㄴ02(label)
            :
      ㄴ10(label)
    ㄴphrases
  ㄴF02
     :
  ㄴM02
  ㄴcalib.txt
'''

for folder in os.listdir(IMG_PATH):
    folder_path = os.path.join(IMG_PATH, folder)
    if os.path.isdir(folder_path):
        # phrases 폴더 내의 모든 jpg 파일에 대해 반복
        p_folder = os.path.join(folder_path, "phrases")
        if os.path.exists(p_folder):
            for label_folder in os.listdir(p_folder):
                label_folder_path = os.path.join(p_folder, label_folder)
                if os.path.isdir(label_folder_path):
                    for instance_folder in os.listdir(label_folder_path):
                        instance_folder_path = os.path.join(label_folder_path, instance_folder)
                        if os.path.isdir(instance_folder_path):
                            for file_name in os.listdir(instance_folder_path):
                                if file_name.endswith(".jpg"):
                                    image_path = os.path.join(instance_folder_path, file_name)
                                    process_image(image_path)


In [ ]:

for folder in os.listdir(IMG_PATH):
    folder_path = os.path.join(IMG_PATH, folder)
    if os.path.isdir(folder_path):
        # phrases 폴더 내의 모든 jpg 파일에 대해 반복
        p_folder = os.path.join(folder_path, "words")
        if os.path.exists(p_folder):
            for label_folder in os.listdir(p_folder):
                label_folder_path = os.path.join(p_folder, label_folder)
                if os.path.isdir(label_folder_path):
                    for instance_folder in os.listdir(label_folder_path):
                        instance_folder_path = os.path.join(label_folder_path, instance_folder)
                        if os.path.isdir(instance_folder_path):
                            for file_name in os.listdir(instance_folder_path):
                                if file_name.endswith(".jpg"):
                                    image_path = os.path.join(instance_folder_path, file_name)
                                    process_image(image_path)
